In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
sidetbDF = pd.read_csv('./project/side_table.csv', index_col=0)
sidetbDF['image'] 

0       ./static/img/living/side_table/side_table1.png
1       ./static/img/living/side_table/side_table2.png
2       ./static/img/living/side_table/side_table3.png
3       ./static/img/living/side_table/side_table4.png
4       ./static/img/living/side_table/side_table5.png
                            ...                       
94     ./static/img/living/side_table/side_table98.png
95     ./static/img/living/side_table/side_table99.png
96    ./static/img/living/side_table/side_table100.png
97    ./static/img/living/side_table/side_table101.png
98    ./static/img/living/side_table/side_table102.png
Name: image, Length: 99, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(sidetbDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

sidetbDF['nouns'] = box

100%|██████████| 99/99 [00:17<00:00,  5.77it/s]


In [5]:
sidetbDF.head(1)

,brand,item,price,image,link,review,nouns
0,먼데이하우스,순수원목 A사이드테이블 3색,"18,900",./static/img/living/side_table/side_table1.png,https://ohou.se/productions/388715/selling?aff...,['침대 옆 협탁 겸 화장대로 사용하려고 구매했어요. 본가에 블랙으로 사서 사용했는...,침대 옆 협탁 겸 화장 사용 구매 본가 블랙 사서 사용 자취 집 화이트 모로 사용 ...


In [6]:
# 중간 저장
sidetbDF.to_csv('sidetbDF.csv')

In [7]:
sidetbDF = pd.read_csv('./sidetbDF.csv', index_col=0)
sidetbDF.head(1)

,brand,item,price,image,link,review,nouns
0,먼데이하우스,순수원목 A사이드테이블 3색,"18,900",./static/img/living/side_table/side_table1.png,https://ohou.se/productions/388715/selling?aff...,['침대 옆 협탁 겸 화장대로 사용하려고 구매했어요. 본가에 블랙으로 사서 사용했는...,침대 옆 협탁 겸 화장 사용 구매 본가 블랙 사서 사용 자취 집 화이트 모로 사용 ...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(sidetbDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_sidetb = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_sidetb

,가갹,가게,가격,가계부,가공,가굑,가구,가구라,가기,가까이,...,흥건,흰색,힌거,힐것,힐링,힐수,힘껏,힘들엇어,힙하,힙해
0,0,0,19,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,16,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,20,0,1,0,1,0,1,0,...,0,2,0,0,0,0,0,0,0,0
3,0,0,14,0,0,0,2,0,1,0,...,0,2,0,0,0,0,0,0,0,0
4,0,0,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,17,0,0,0,11,0,0,0,...,0,1,0,0,0,0,0,0,0,0
95,0,0,14,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,15,0,0,0,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,5,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# dtm 파일 저장
dtm_sidetb.to_csv('dtm_sidetb.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

sidetbDF = pd.read_csv('./sidetbDF.csv', index_col=0)
dtm_sidetb = pd.read_csv('./dtm_sidetb.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(sidetbDF['brand'])
item_list = np.array(sidetbDF['item'])
price_list = np.array(sidetbDF['price'])
image_list = np.array(sidetbDF['image'])

result_cosine = make_cosine(dtm_sidetb, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

한샘 클린트 어반 사이드테이블_라운드 148,000 ./static/img/living/side_table/side_table75.png


In [12]:
# 위의 코드 함수화
def find_sidetable(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    sidetbDF = pd.read_csv('./sidetbDF.csv', index_col=0)
    dtm_sidetb = pd.read_csv('./dtm_sidetb.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(sidetbDF['brand'])
    item_list = np.array(sidetbDF['item'])
    price_list = np.array(sidetbDF['price'])
    image_list = np.array(sidetbDF['image'])

    result_cosine = make_cosine(dtm_sidetb, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]